In [1]:
!pip install -q transformers torch

In [7]:
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.5 MB/s eta 0:00:00


In [8]:
!pip install -q accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.4 MB/s eta 0:00:00


In [9]:
!pip install -q transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.0 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path_csv = '/content/drive/My Drive/Colab Notebooks/Ch6_Paraphrasing/sentiment_paraphrasing_corpus_curated.csv'

df = pd.read_csv(file_path_csv)
df

,id,sentence_orig,paraphrase_pos,paraprhase_neg,sentence_orig_label,sentence_orig_score,paraphrase_pos_label,paraphrase_pos_score,paraprhase_neg_label,paraprhase_neg_score,pos_is_paraphrase,neg_is_paraphrase
0,1,We are going to shopping mall.,We are going to shopping mall and I like it.,We are going to shopping mall and I hate it.,NEU,0.914790,POS,0.991477,NEG,0.975471,0.796854,0.578557
1,2,We are going to shopping.,It would be nice if we can go shopping.,"We are not go shopping again, I hope not.",NEU,0.887519,POS,0.922553,NEG,0.953186,0.693683,0.600350
2,3,We are going to flea market,"We are going to flea market, which is really a...","We are going to flea market, what a bummer.",NEU,0.821980,POS,0.984940,NEG,0.918379,0.793140,0.782593
3,4,We are going to buy some groceries.,There is no better thing than bying some groce...,We are going to buy groceries which is annoying.,NEU,0.922462,POS,0.992699,NEG,0.966358,0.596304,0.772327
4,5,We buy vegetables every day.,We like to buy fresh and delicates vegetables ...,"We don't buy vegetables at all, it is disgusting.",NEU,0.955400,POS,0.786090,NEG,0.978400,0.802271,0.699377
5,6,We are going to public swimming pool.,We are going to public swimming pool and I lik...,We are going to public swimming pool and I hat...,NEU,0.945938,POS,0.990365,NEG,0.976949,0.881950,0.709731
6,7,We are going to swimm.,It would be nice if we can go swimm.,"No, we don't go swimm again.",NEU,0.812850,POS,0.942458,NEG,0.909584,0.746288,0.739073
7,8,We are going to garage,"We are going to garage, finally drive the car.","We are going to ugly, dark garage.",NEU,0.926212,POS,0.570268,NEU,0.840668,0.741850,0.681821
8,9,We are going to buy some stocks.,We are going to buy some profitable stock.,We are going to buy some lousy stocks.,NEU,0.959553,POS,0.725651,NEG,0.974478,0.858337,0.792223
9,10,We buy candies every day.,"We buy candies every day, its yummy.","We don't buy candies everry day, it sucks.",NEU,0.874354,POS,0.988138,NEG,0.975274,0.886961,0.769451


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd

# Load pre-trained model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

class ParaphraseDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, dataframe, max_length=512):
        self.examples = []
        for _, row in dataframe.iterrows():
            # Positive paraphrase example
            self.examples.append({
                "input_text": ">>pos<< paraphrase: " + row['sentence_orig'] + " </s>",
                "target_text": row['paraphrase_pos'] + " </s>"
            })
            # Negative paraphrase example
            self.examples.append({
                "input_text": ">>neg<< paraphrase: " + row['sentence_orig'] + " </s>",
                "target_text": row['paraprhase_neg'] + " </s>"
            })
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        input_encoding = tokenizer(
            example['input_text'],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = tokenizer(
            example['target_text'],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": input_encoding.input_ids.flatten(),
            "attention_mask": input_encoding.attention_mask.flatten(),
            "labels": target_encoding.input_ids.flatten()
        }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

train_dataset = ParaphraseDataset(tokenizer, df)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/p